## SQL Databases

In [1]:
## Database content extraction
from langchain_community.utilities import SQLDatabase
from langchain_community.document_loaders import SQLDatabaseLoader

In [2]:
## Method 1: SQLDatabase Utility
db = SQLDatabase.from_uri("sqlite:///data/databases/company.db")

## get database info
print(f"Tables: {db.get_usable_table_names()}")
print(f"\nTables:")
print(db.get_table_info())

Tables: ['employees', 'projects']

Tables:

CREATE TABLE employees (
	id INTEGER, 
	name TEXT, 
	role TEXT, 
	department TEXT, 
	salary REAL, 
	PRIMARY KEY (id)
)

/*
3 rows from employees table:
id	name	role	department	salary
1	John Doe	Senior Developer	Engineering	95000.0
2	Jane Smith	Data Scientist	Analytics	105000.0
3	Mike Johnson	Product Manager	Product	110000.0
*/


CREATE TABLE projects (
	id INTEGER, 
	name TEXT, 
	status TEXT, 
	budget REAL, 
	lead_id INTEGER, 
	PRIMARY KEY (id)
)

/*
3 rows from projects table:
id	name	status	budget	lead_id
1	RAG Implementation	Active	150000.0	1
2	Data Pipeline	Completed	80000.0	2
3	Customer Portal	Planning	200000.0	3
*/


In [ ]:
from typing import List
import sqlite3
from langchain_core.documents import Document

def sql_to_documents(db_path: str) -> List[Document]:
    """Convert SQL Database to documents with context"""
    
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    
    documents = []
    
    cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cur.fetchall()
    
    
    for table in tables:
        table_name = table[0]
        
        # Get table schema
        try:
            cur.execute(f"PRAGMA table_info({table_name});")
        except sqlite3.Error as e:
            print(f"An error occurred: {e}")
        columns = cur.fetchall()
        column_names = [col[1] for col in columns]
        
        # get table data
        cur.execute(f"SELECT * FROM {table_name}")
        rows = cur.fetchall()
        
        # Create table overview document
        table_content = f"Table: {table_name}\n"
        table_content += f"Columns: {', '.join(column_names)}\n"
        table_content += f"Total Records: {len(rows)}\n\n"
        
        # Add sample records        
        table_content += f"Sample Records:\n"
        for row in rows[:5]:  # First 5 rows only
            record = dict(zip(column_names, row))
            table_content += f"{record}\n"
        doc = Document(
            page_content=table_content,
            metadata={
                "source": db_path, 
                "table_name": table_name,
                "num_records": len(rows),
                "data_type": 'sql_table'
            }
        )
        documents.append(doc)
    return documents

In [20]:
document = sql_to_documents("data/databases/company.db")
document

[Document(metadata={'source': 'data/databases/company.db', 'table_name': 'employees', 'num_records': 4, 'data_type': 'sql_table'}, page_content="Table: employees\nColumns: id, name, role, department, salary\nTotal Records: 4\n\nSample Records:\n{'id': 1, 'name': 'John Doe', 'role': 'Senior Developer', 'department': 'Engineering', 'salary': 95000.0}\n{'id': 2, 'name': 'Jane Smith', 'role': 'Data Scientist', 'department': 'Analytics', 'salary': 105000.0}\n{'id': 3, 'name': 'Mike Johnson', 'role': 'Product Manager', 'department': 'Product', 'salary': 110000.0}\n{'id': 4, 'name': 'Sarah Williams', 'role': 'DevOps Engineer', 'department': 'Engineering', 'salary': 98000.0}\n"),
 Document(metadata={'source': 'data/databases/company.db', 'table_name': 'projects', 'num_records': 4, 'data_type': 'sql_table'}, page_content="Table: projects\nColumns: id, name, status, budget, lead_id\nTotal Records: 4\n\nSample Records:\n{'id': 1, 'name': 'RAG Implementation', 'status': 'Active', 'budget': 150000.